In [ ]:
import cv2 as cv
import numpy as np
import math
import collections

In [ ]:
QTY = np.array([[16, 11, 10, 16, 24, 40, 51, 61],  # luminance quantization table
                [12, 12, 14, 19, 26, 48, 60, 55],
                [14, 13, 16, 24, 40, 57, 69, 56],
                [14, 17, 22, 29, 51, 87, 80, 62],
                [18, 22, 37, 56, 68, 109, 103, 77],
                [24, 35, 55, 64, 81, 104, 113, 92],
                [49, 64, 78, 87, 103, 121, 120, 101],
                [72, 92, 95, 98, 112, 100, 103, 99]])
QTC = np.array([[17, 18, 24, 47, 99, 99, 99, 99],  # chrominance quantization table
                [18, 21, 26, 66, 99, 99, 99, 99],
                [24, 26, 56, 99, 99, 99, 99, 99],
                [47, 66, 99, 99, 99, 99, 99, 99],
                [99, 99, 99, 99, 99, 99, 99, 99],
                [99, 99, 99, 99, 99, 99, 99, 99],
                [99, 99, 99, 99, 99, 99, 99, 99],
                [99, 99, 99, 99, 99, 99, 99, 99]])

In [ ]:
def RGBtoYCbCr(R,G,B):
  
  Y=0.299*R + 0.587*G + 0.114*B
  Cb=-0.1687*R - 0.3313*G + 0.5*B + 128
  Cr=0.5*R - 0.4187*G - 0.0813*B + 128

  return Y,Cb,Cr

In [ ]:
def DCT(matrix):
  # dct will store the discrete cosine transform
  pi = 3.142857
  dct = np.zeros((8,8))
  m=8
  n=8
  for i in range(8):
    for j in range(8):
      # ci and cj depends on frequency as well as
      # number of row and columns of specified matrix
      if (i == 0):
        ci = 1 / (m ** 0.5)
      else:
        ci = (2 / m) ** 0.5
      if (j == 0):
        cj = 1 / (n ** 0.5)
      else:
        cj = (2 / n) ** 0.5
 
      # sum will temporarily store the sum of
      # cosine signals
      sum = 0
      for k in range(m):
        for l in range(n):
          dct1 = matrix[k][l] * math.cos((2 * k + 1) * i * pi / (2 * m)) * math.cos((2 * l + 1) * j * pi / (2 * n))
          sum = sum + dct1
          dct[i][j] = ci * cj * sum
  return dct

In [ ]:
def DCT1(matrix):
  T= np.array([[ 0.354,  0.354,  0.354,  0.354,  0.354,  0.354,  0.354,  0.354],
    [ 0.49,   0.416,  0.278,  0.098, -0.098, -0.278, -0.416, -0.49 ],
    [ 0.462,  0.191, -0.191, -0.462, -0.462, -0.191 , 0.191,  0.462],
    [ 0.416, -0.098, -0.49,  -0.278,  0.278,  0.49,   0.098, -0.416],
    [ 0.354, -0.354, -0.354,  0.354,  0.354, -0.354, -0.354,  0.354],
    [ 0.278, -0.49,   0.098,  0.416, -0.416, -0.098,  0.49,  -0.278],
    [ 0.191, -0.462,  0.462, -0.191, -0.191,  0.462, -0.462,  0.191],
    [ 0.098, -0.278,  0.416, -0.49,   0.49,  -0.416,  0.278, -0.098]])
  Tt=T.transpose()
  temp=matrix@Tt
  dct=T@temp
  return dct

In [ ]:
def QTlum(matrix):
  
  QTY = np.array([[16, 11, 10, 16, 24, 40, 51, 61],  # luminance quantization table
                [12, 12, 14, 19, 26, 48, 60, 55],
                [14, 13, 16, 24, 40, 57, 69, 56],
                [14, 17, 22, 29, 51, 87, 80, 62],
                [18, 22, 37, 56, 68, 109, 103, 77],
                [24, 35, 55, 64, 81, 104, 113, 92],
                [49, 64, 78, 87, 103, 121, 120, 101],
                [72, 92, 95, 98, 112, 100, 103, 99]])

  qtlum=(matrix/QTY)
  
  return qtlum.astype(int)

In [ ]:
def QTchr(matrix):

  QTC = np.array([[17, 18, 24, 47, 99, 99, 99, 99],  # chrominance quantization table
                [18, 21, 26, 66, 99, 99, 99, 99],
                [24, 26, 56, 99, 99, 99, 99, 99],
                [47, 66, 99, 99, 99, 99, 99, 99],
                [99, 99, 99, 99, 99, 99, 99, 99],
                [99, 99, 99, 99, 99, 99, 99, 99],
                [99, 99, 99, 99, 99, 99, 99, 99],
                [99, 99, 99, 99, 99, 99, 99, 99]])
  
  qtchr=matrix/QTC
  
  return qtchr.astype(int)

In [ ]:
def zigzag(matrix):
  
  h = 0
  v = 0
  v_min = 0
  h_min = 0
  v_max = matrix.shape[0]
  h_max = matrix.shape[1]
  i = 0
  output = np.zeros((v_max * h_max))

  while (v < v_max) and (h < h_max):
    if ((h + v) % 2) == 0:  # going up
      if v == v_min:
        output[i] = int(matrix[v, h])  # first line
        if h == h_max:
          v = v + 1
        else:
          h = h + 1
          i = i + 1
      elif (h == h_max - 1) and (v < v_max):  # last column
        output[i] = int(matrix[v, h])
        v = v + 1
        i = i + 1
      elif (v > v_min) and (h < h_max - 1):  # all other cases
        output[i] = int(matrix[v, h])
        v = v - 1
        h = h + 1
        i = i + 1
    else:  # going down
      if (v == v_max - 1) and (h <= h_max - 1):  # last line
        output[i] = int(matrix[v, h])
        h = h + 1
        i = i + 1
      elif h == h_min:  # first column
        output[i] = int(matrix[v, h])
        if v == v_max - 1:
          h = h + 1
        else:
          v = v + 1
          i = i + 1
      elif (v < v_max - 1) and (h > h_min):  # all other cases
         output[i] = int(matrix[v, h])
         v = v + 1
         h = h - 1
         i = i + 1
      if (v == v_max - 1) and (h == h_max - 1):  # bottom right element
         output[i] = int(matrix[v, h])
         break

  return output

In [ ]:
def rl_encoding(zz):
  rlen=list()
  rl=0
  dc0=0
  c=0
  for i in range(len(zz)):
    if(i%64==63):
      eob=('EOB',)
      rlen.append(eob)
      c=0
      rl=0
    if i%64==0:
      dc1=zz[i]-dc0
      dc=(int(dc1).bit_length(),int(dc1))
      #rlen=rlen.append(int(zz[0]).bit_length())
      #rlen=rlen.append(int(zz[i]))
      dc0=zz[i]
      rlen.append(dc)
      
    elif zz[i]==0:
      rl=rl+1
      
    else:
      ac=(rl,int(zz[i]).bit_length(),int(zz[i]))
      rlen.append(ac)
      rl=0
      
  return rlen

In [ ]:
def Frequency(encoded):
  Output = collections.defaultdict(int)
  
  for elem in encoded:
          
    Output[elem[0:2]] += 1
            
        
  return Output

In [ ]:


# A Huffman Tree Node
import heapq

class node:
	def __init__(self, freq, symbol, left=None, right=None):
		# frequency of symbol
		self.freq = freq

		# symbol name (character)
		self.symbol = symbol

		# node left of current node
		self.left = left

		# node right of current node
		self.right = right

		# tree direction (0/1)
		self.huff = ''
		
	def __lt__(self, nxt):
		return self.freq < nxt.freq
		


# created Huffman tree

def huffmann(a):
  huffmann_codes = collections.defaultdict(int)

  def getCodes(node, val=''):
    
    # huffman code for current node
    newVal = val + str(node.huff)

    # if node is not an edge node
    # then traverse inside it
    if(node.left):
      getCodes(node.left, newVal)
    if(node.right):
      getCodes(node.right, newVal)

      # if node is edge node then
      # display its huffman code
    if(not node.left and not node.right):
                      huffmann_codes[node.symbol] = newVal
                      
                     
                              





  # list containing unused nodes
  nodes = []

  for k in a:
    heapq.heappush(nodes, node(a[k], k[0:2]))
    


  #Creating Tree 
  while len(nodes) > 1:
    
    # sort all the nodes in ascending order
    # based on their frequency
    left = heapq.heappop(nodes)
    right = heapq.heappop(nodes)

    # assign directional value to these nodes
    left.huff = 0
    right.huff = 1

    # combine the 2 smallest nodes to create
    # new node as their parent
    newNode = node(left.freq+right.freq, left.symbol+right.symbol, left, right)

    heapq.heappush(nodes, newNode)


  getCodes(nodes[0])
  return huffmann_codes
  #print(huffmann_codes)



In [ ]:
def binary(k):
  st = bin(k)
  if k < 0:
          for i in range(len(st)):
              if st[i] == "0" :
                st = st[0:i]+"1"+st[i+1:]
              else: st = st[0:i]+"0"+st[i+1:]
          return st[3:]
        

  else:
          return st[2:]

In [ ]:
def compress_data(encoded,huffmann_codes):
  
  compress = str()
  for elem in encoded:

    if len(elem) <=2 :
    
      if elem == ("END",):
        compress+=huffmann_codes[elem]
      else:
        compress += huffmann_codes[elem[0:2]]

    else:
      compress+=huffmann_codes[elem[0:2]] + binary(elem[2])
    
  return compress

In [ ]:
def Inbytes(compress):
  y = len(compress)%8
  if y!=0:
    compress += "0"*(8-y)

  Byte = bytes()
  num=0
  for i in range(0, len(compress) - 1, 8):
          num = int(compress[i:i+8],base=2)
        
          chr = num.to_bytes(1, 'little')
          Byte += chr
  return Byte

In [ ]:
img = cv.imread("/content/lena_colored_256.bmp")

h=img.shape[0]
w=img.shape[1]
channel=img.shape[2]
print(img.shape)
R=img[:,:,0]
G=img[:,:,1]
B=img[:,:,2]

img[:,:,0]=0
img[:,:,1]=0
img[:,:,2]


#cv.imwrite("hdsgfhjgjhgjh.jpg",img[:,:,0])
imgYCC = cv.cvtColor(img, cv.COLOR_BGR2YCR_CB)
#Y=imgYCC[:,:,2]
#imgYCC=np.zeros((h,w,channel))
Y=imgYCC[:,:,0]
Cr=imgYCC[:,:,1]
Cb=imgYCC[:,:,2]
#plt.imshow(Cb)
#Y,Cb,Cr=RGBtoYCbCr(R,G,B)
print(Y)
print(Cb)
print(Cr)

Y=Y-128
Cb=Cb-128
Cr=Cr-128

ypadh=Y.shape[0] % 8
ypadv=Y.shape[1] % 8
print(ypadh)
print(ypadv)
Y=np.pad(Y, pad_width=((0, (8-ypadh)%8), (0, (8-ypadv)%8)))
Cb=np.pad(Y, pad_width=((0, (8-ypadh)%8), (0, (8-ypadv)%8)))
Cr=np.pad(Y, pad_width=((0, (8-ypadh)%8), (0, (8-ypadv)%8)))

h=Y.shape[0]
w=Y.shape[1]

imgdctY=np.zeros((Y.shape[0],Y.shape[1]))
print(Y.shape)
imgqtY=np.zeros((Y.shape[0],Y.shape[1]))
imgqtCb=np.zeros((Y.shape[0],Y.shape[1]))
imgqtCr=np.zeros((Y.shape[0],Y.shape[1]))
#print(Cr-Cb)
zigzagY=list()
zigzagCb=list()
zigzagCr=list()
rlY=list()
rlCb=list()
rlCr=list()
dc=list()
dc.append(0)
for i in range(0, Y.shape[0], 8):
  for j in range(0, Y.shape[1], 8):
    
    blockY=Y[i : i+8 , j : j+8]
    blockCb=Cb[i : i+8 , j : j+8]
    blockCr=Cr[i : i+8 , j : j+8]

    ##Blockwise DCT Calcuilation
    dctblockY=DCT1(blockY)
    dctblockCb=DCT1(blockCb)
    dctblockCr=DCT1(blockCr)

    imgdctY[i : i+8 , j : j+8]=dctblockY
    #print(dctblock)

    ##Blockwise Quantisation
    qtblockY=QTlum(dctblockY)
    qtblockCb=QTchr(dctblockCb)
    qtblockCr=QTchr(dctblockCr)
    imgqtY[i : i+8 , j : j+8]=qtblockY
    imgqtCb[i : i+8 , j : j+8]=qtblockCb
    imgqtCr[i : i+8 , j : j+8]=qtblockCr
    #print(qtblock)
    #print()

    ##ZIGZAG Traversal Blockwise
    zY=zigzag(qtblockY)
    zCb=zigzag(qtblockCb)
    zCr=zigzag(qtblockCr)
    zigzagY.extend(zY)
    zigzagCb.extend(zCb)
    zigzagCr.extend(zCr)


    #rl[0]=rl[0]-dc[0]
    #dc=dc1
    
    ##RUNLENGHT ENCODING
    
rlY=rl_encoding(zigzagY)
rlCb=rl_encoding(zigzagCb)
rlCr=rl_encoding(zigzagCr)
print(imgdctY)
print(imgqtY)
"""
print(zigzagY)
print(zigzagY[0])
"""
print(rlY)
print(rlCb)
#print(rlCr)
#print(imgdctY.shape)

huffmann_y = collections.defaultdict(int)
huffmann_cb = collections.defaultdict(int)
huffmann_cr = collections.defaultdict(int)

Freq_y = Frequency(rlY)
Freq_cb = Frequency(rlCb)
Freq_cr = Frequency(rlCr)


print(Freq_y)
print(Freq_cb)
print(Freq_cr)

huffmann_y = huffmann(Freq_y)
huffmann_cb = huffmann(Freq_cb)
huffmann_cr = huffmann(Freq_cr)

print(huffmann_y)

y_compressed = compress_data(rlY,huffmann_y)
cb_compressed = compress_data(rlCb,huffmann_cb)
cr_compressed = compress_data(rlCr,huffmann_cr)

print(imgqtY)


(256, 256, 3)
[[68 68 68 ... 69 68 64]
 [68 68 68 ... 73 72 63]
 [67 67 67 ... 62 58 49]
 ...
 [26 28 29 ... 41 44 46]
 [25 27 28 ... 44 47 48]
 [26 27 28 ... 46 48 49]]
[[ 90  90  90 ...  89  90  92]
 [ 90  90  90 ...  87  87  92]
 [ 90  90  90 ...  93  95 100]
 ...
 [113 112 112 ... 105 103 102]
 [114 113 112 ... 103 101 101]
 [113 113 112 ... 102 101 100]]
[[241 241 241 ... 244 241 234]
 [241 241 241 ... 250 248 234]
 [241 241 241 ... 232 224 210]
 ...
 [171 174 176 ... 196 201 206]
 [171 173 175 ... 202 207 209]
 [171 172 174 ... 205 209 210]]
0
0
(256, 256)
[[ 1.56394368e+03  6.91713353e-15 -7.70317143e-15 ...  3.63204000e-01
  -3.00546000e-01  3.71700000e-01]
 [ 3.63062400e+00 -1.18904886e-14  1.14452892e-15 ... -4.44024000e-01
  -5.01080000e-02 -4.54132000e-01]
 [ 3.69859200e+00 -1.59494640e-14  5.00233188e-15 ...  2.31760000e-01
  -5.99040000e-02 -1.37974000e-01]
 ...
 [-3.73824000e-01 -2.17652000e-01 -2.67064000e-01 ...  4.43880000e-02
   1.10580000e-01  3.16852000e-01]
 [ 5.0

In [ ]:
print(Freq_y[(53,3)])

0


In [ ]:
Byte_qy = bytes()

Byte_qc = bytes()

for i in range(8):
  for j in range(8):
    Byte_qy += str(QTY[i,j]).encode()
    Byte_qc += str(QTC[i,j]).encode()

bytedata = bytes()

bytedata+=Inbytes(y_compressed)
bytedata+=Inbytes(cb_compressed)
bytedata+=Inbytes(cr_compressed)

huffmann_tables = bytes()
huffmann_tables = str(huffmann_y).encode() + str(huffmann_cb).encode() + str(huffmann_cr).encode()
bytedata = b'\xFFD8'+ b'\xFFC0'+ str((img.shape[0],img.shape[1])).encode()+ b'\xFFDB' + Byte_qy + b'\xFFDB' + Byte_qc + b'\xFFC4' + huffmann_tables  + b'\x00' +bytedata + b'\xFFD9'

print(bytedata)

with open('file.txt','wb') as data: 
      data.write(bytedata)

b'\xffD8\xffC0(256, 256)\xffDB1611101624405161121214192648605514131624405769561417222951878062182237566810910377243555648110411392496478871031211201017292959811210010399\xffDB17182447999999991821266699999999242656999999999947669999999999999999999999999999999999999999999999999999999999999999999999999999\xffC4defaultdict(<class \'int\'>, {(2, 3): \'000000\', (3, -4): \'0000010\', (4, 12): \'00000110\', (4, 9): \'00000111\', (5, 1): \'000010\', (3, 3): \'00001100\', (9, 1): \'0000110100\', (4, -15): \'0000110101\', (4, -12): \'000011011\', (4, -10): \'0000111\', (0, 0): \'0001\', (1, 1): \'001\', (1, 2): \'0100\', (3, -6): \'01010000\', (4, -14): \'0101000100\', (5, 16): \'01010001010\', (10, 1): \'01010001011\', (5, -16): \'010100011\', (3, 5): \'0101001\', (2, -2): \'010101\', (3, -5): \'0101100\', (3, 4): \'0101101\', (3, 2): \'010111\', (0, 2): \'0110\', (4, 1): \'011100\', (1, 4): \'0111010\', (7, 1): \'01110110\', (5, -18): \'01110111000\', (5, -17): \'01110111001\', (5, -19): \'011